In [1]:
1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1724708581010_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [3]:
from campaign_utils.common_funtion import CommonFunction


role_arn='arn:aws:iam::336913029742:role/AWS-Airflow-Deploy-Role'

sts = boto3.client('sts')
response = sts.assume_role(
    RoleArn=role_arn,
    RoleSessionName='my-random-session-name',
    DurationSeconds=3600
)

credentials = response['Credentials']

# Create a session with the assumed role's credentials
session = Session(
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken']
)

# Use the session to create an S3 resource
s3_resource = session.resource('s3')



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import io
import os
import re
import glob
import pandas as pd
import numpy as np

from datetime import datetime
import datetime as dt
import time
import boto3

import zipfile
import pytz

from datetime import date, timedelta
import gzip
import pandas as pd
from pandas.io.json import json_normalize 
import json
import gnupg
from io import StringIO
import boto3
from boto3.session import Session
import paramiko
import os
import requests
import logging
import os
import typing
from os.path import join, dirname
import time
import logging
from paramiko import SFTPClient, SFTPFile, Message, SFTPError, Transport
from paramiko.sftp import CMD_STATUS, CMD_READ, CMD_DATA
from campaign_utils.common_funtion import CommonFunction

logger = logging.getLogger('demo')
S3_RES = CommonFunction.get_aws_resource_obj(resource_name='s3')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
class _SFTPFileDownloader:
    """
    Helper class to download large file with paramiko sftp client with limited number of concurrent requests.
    """

    _DOWNLOAD_MAX_REQUESTS = 48
    _DOWNLOAD_MAX_CHUNK_SIZE = 0x8000

    def __init__(self, f_in: SFTPFile, f_out: typing.BinaryIO, callback=None):
        self.f_in = f_in
        self.f_out = f_out
        self.callback = callback

        self.requested_chunks = {}
        self.received_chunks = {}
        self.saved_exception = None

    def download(self):
        file_size = self.f_in.stat().st_size
        requested_size = 0
        received_size = 0

        while True:
            # send read requests
            while len(self.requested_chunks) + len(self.received_chunks) < self._DOWNLOAD_MAX_REQUESTS and \
                    requested_size < file_size:
                chunk_size = min(self._DOWNLOAD_MAX_CHUNK_SIZE, file_size - requested_size)
                request_id = self._sftp_async_read_request(
                    fileobj=self,
                    file_handle=self.f_in.handle,
                    offset=requested_size,
                    size=chunk_size
                )
                self.requested_chunks[request_id] = (requested_size, chunk_size)
                requested_size += chunk_size

            # receive blocks if they are available
            # note: the _async_response is invoked
            self.f_in.sftp._read_response()
            self._check_exception()

            # write received data to output stream
            while True:
                chunk = self.received_chunks.pop(received_size, None)
                if chunk is None:
                    break
                _, chunk_size, chunk_data = chunk
                self.f_out.write(chunk_data)
                if self.callback is not None:
                    self.callback(chunk_data)

                received_size += chunk_size

            # check transfer status
            if received_size >= file_size:
                break

            # check chunks queues
            if not self.requested_chunks and len(self.received_chunks) >= self._DOWNLOAD_MAX_REQUESTS:
                raise ValueError("SFTP communication error. The queue with requested file chunks is empty and"
                                 "the received chunks queue is full and cannot be consumed.")

        return received_size

    def _sftp_async_read_request(self, fileobj, file_handle, offset, size):
        sftp_client = self.f_in.sftp

        with sftp_client._lock:
            num = sftp_client.request_number

            msg = Message()
            msg.add_int(num)
            msg.add_string(file_handle)
            msg.add_int64(offset)
            msg.add_int(size)

            sftp_client._expecting[num] = fileobj
            sftp_client.request_number += 1

        sftp_client._send_packet(CMD_READ, msg)
        return num

    def _async_response(self, t, msg, num):
        if t == CMD_STATUS:
            # save exception and re-raise it on next file operation
            try:
                self.f_in.sftp._convert_status(msg)
            except Exception as e:
                self.saved_exception = e
            return
        if t != CMD_DATA:
            raise SFTPError("Expected data")
        data = msg.get_string()

        chunk_data = self.requested_chunks.pop(num, None)
        if chunk_data is None:
            return

        # save chunk
        offset, size = chunk_data

        if size != len(data):
            raise SFTPError(f"Invalid data block size. Expected {size} bytes, but it has {len(data)} size")
        self.received_chunks[offset] = (offset, size, data)

    def _check_exception(self):
        """if there's a saved exception, raise & clear it"""
        if self.saved_exception is not None:
            x = self.saved_exception
            self.saved_exception = None
            raise x


def download_file(sftp_client: SFTPClient, remote_path: str, local_path: str, callback=None):
    """
    Helper function to download remote file via sftp.
    It contains a fix for a bug that prevents a large file downloading with :meth:`paramiko.SFTPClient.get`
    Note: this function relies on some private paramiko API and has been tested with paramiko 2.7.1.
          So it may not work with other paramiko versions.
    :param sftp_client: paramiko sftp client
    :param remote_path: remote file path
    :param local_path: local file path
    :param callback: optional data callback
    """
    remote_file_size = sftp_client.stat(remote_path).st_size

    with sftp_client.open(remote_path, 'rb') as f_in, open(local_path, 'wb') as f_out:
        _SFTPFileDownloader(
            f_in=f_in,
            f_out=f_out,
            callback=callback
        ).download()

    local_file_size = os.path.getsize(local_path)
    if remote_file_size != local_file_size:
        raise IOError(f"file size mismatch: {remote_file_size} != {local_file_size}")


def main():

    start_time = time.time()
    # public sftp server
    host = 'DOTS.datalabusa.com'
    port = 22
    username = 'ex_Avant'
    password = 'Et#h@ut8Ec2F'
    
    ### Ignore file list    
    s3_resource = boto3.resource("s3") 
    my_bucket = s3_resource.Bucket('avant-partner01-landing')
    list1 = ['DL_AVT_REMAILED_202409_20240826.csv.pgp']
    
#     for my_bucket_object in my_bucket.objects.filter(Prefix='datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410'):
#         a = my_bucket_object.key.replace('datalabs/drop/interim/','')
#         list1.append(a)

#     for my_bucket_object in my_bucket.objects.filter(Prefix='datalabs/drop/interim/DL_AVT_INTERIMFILE2_202410'):
#         a = my_bucket_object.key.replace('datalabs/drop/interim/','')
#         list1.append(a)

    ### Getting file list
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    ssh.connect(hostname=host,port=port,username=username,password=password)
    sftp = ssh.open_sftp()

    file_list = sftp.listdir('From_Datalab')
    campaign_month = '202409' 
    campaign_files = [file for file in file_list if campaign_month in file and file in list1]
    
    ssh.close()
    sftp.close()
    
    
    for file in campaign_files:
    
        remote_file_path = 'From_Datalab/{}'.format(file)
        local_file_path = '/tmp/{}'.format(file)

        transport = Transport((host, port))
        transport.set_keepalive(30)
        transport.connect(
            username=username,
            password=password,
        )

        with SFTPClient.from_transport(transport) as sftp_client:
            progress_size = 0
            total_size = 0
            step_size = 4 * 1024 * 1024

            def progress_callback(data):
                nonlocal progress_size, total_size
                progress_size += len(data)
                total_size += len(data)
                while progress_size >= step_size:
                    logger.info(f"{total_size // (1024 ** 2)} MB has been downloaded")
                    progress_size -= step_size

            download_file(
                sftp_client=sftp_client,
                remote_path=remote_file_path,
                local_path=local_file_path,
                callback=progress_callback
            )
        download_time = time.time() - start_time
        print(f"File downloaded successfully to {local_file_path} in {download_time:.2f} seconds")

        # s4 upload
        s3_bucket_name = 'avant-partner01-landing'
        s3_object_key = 'datalabs/drop/mailed/{file}'.format(
        campaign = campaign_month,
        file = file
        )

        S3_RES.meta.client.upload_file(
            local_file_path,
            s3_bucket_name,
            s3_object_key,
            ExtraArgs={
                "ACL": "bucket-owner-full-control",
                "ServerSideEncryption": "AES256"
            }
        )

        # Clean up local file
    
if __name__ == 'builtins':
    logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")
    main()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

File downloaded successfully to /tmp/DL_AVT_REMAILED_202409_20240826.csv.pgp in 5.87 seconds

In [21]:
args = {'ENV': 'prod', 'REGION': 'us-east-2', 'account_id': '235352882622', 'PARTNER_NAME': 'avant', 'emr_details': {}, 'AIRFLOW_PATH': '/usr/local/airflow', 'SQL_BASE_PATH': '/usr/local/airflow/test', 'DREMIO_ENV': 'prod', 'PARTNER_ID': 'partner01', 'measurement_date': '2024-08-25', 'YEAR': '2024', 'MONTH': '08', 'DATE': '25', 'YYYY': '2024', 'MM': '08', 'DD': '25', 'date': '2024-08-25', 'date_time': '2024-08-25 18:32:24', 'smtp_details': {'EMAIL_FROM': 'campaign-airflow-prod@partner01.data.avant.com'}, 'vault_details': {'vault_url': 'https://vault.shared.partner01.data.avant.com', 'meta_model': 'app/campaign/meta_model/prod', 'ec2_metadata_url': 'http://169.254.169.254', 'group_role': 'airflow-role', 'ec2_role': 'avant-campaign-dev-ecs-task-role', 'emr_role': 'campaign_emr_instance_role', 'vault_meta_base_url': '{base}/latest/meta-data/iam/security-credentials/{role}', 'encryption_details': 'app/landing_zone_encryption/acxiom/2024', 'dremio_jdbc_user': 'app/dremio/jdbc_prod_sa', 'dremio_jdbc_url': 'jdbc:dremio:direct=dremio-int-prod.services.global.avant.com:31010', 'dremio_url': 'https://dremio-prod.services.global.avant.com', 'dremio_vault_path': 'user/devops/dremio/sa_prod_user', 'garden_vault_path': 'user/devops/garden/campaign_garden_scoring', 'smtp_vault_path': 'user/devops/smtp', 'neustar_vault_path': 'app/neustar', 'approval_vault_path': 'app/campaign/approval/prod'}, 'sqs_details': {'encrypt_decrypt_sqs_details': {'sqs_name': '{}_{}_{}_encrypt_decrypt_queue', 'sqs_url': 'https://us-east-2.queue.amazonaws.com/235352882622/DirectMail_2_3_encrypt_decrypt_queue.fifo'}, 'selection_eligibility_sqs_details': {'sqs_name': '{}_{}_{}_selection_eligibility_queue', 'sqs_url': 'https://sqs.us-east-2.amazonaws.com/235352882622/{}.fifo'}, 'selection_desirability_sqs_details': {'sqs_name': '{}_{}_{}_selection_desirability_queue', 'sqs_url': 'https://sqs.us-east-2.amazonaws.com/235352882622/{}.fifo'}}, 's3_details': {'config_file_bucket': 'avantcamp-partner01-prod-common-bucket', 'config_file_names': {'parameter_data': 'common/parameters.cfg', 'feed_schema_tracker': 'common/feed_schema.cfg'}, 'campaign_input_file_prefix_history': 'campaign-user-input-files_history', 'campaign_input_file_prefix': 'campaign-user-input-files', 'campaign_input_file_name': 'campaign - User Input - Template.xlsx', 'campaign_dynamic_dag_file_prefix': 'campaign-system-input-files', 'campaign_dynamic_dag_file_name': 'campaign_dynamic_dag_config.json'}, 'schema_name': 'campaign_config', 'campaign_driver_table_name': 'campaign_driver', 'selection_group_tname': 'selection_group_rule_config', 'uow_dependency_table_name': 'uow_dependency', 'parent_config_id': '1', 'campaign_driver_config_table_name': 'campaign_driver_config', 'common_config_table_name': 'common_system_config', 'airflow_path': '/usr/local/airflow', 'state_dict': {'state_id': {1: 'started', 2: 'inprogress', 3: 'completed', 4: 'failed'}, 'message': {1: 'submitted PySpark job to EMR', 2: 'PySpark job running in EMR', 3: 'Completed PySpark job successfully'}}, 'dremio_table_details': [{'table_name': 'table_name_1', 'table_path': 'path1'}, {'table_name': 'table_name_2', 'table_path': 'path2'}], 'executor_instances': '60', 'sql_base_path': '/usr/local/airflow/sql/{}/{}/{}.sql', 'dremio_security_table_name': 'avant.prod.campaign_abc.campaign_config.campaign_dremio_masking_table', 'local_path': '/usr/local/airflow', 'campaign_type_config_id': '2', 'campaign_name': 'Direct Mail', 'dag_name': 'Direct_Mail_2_2_first_mail_ingestion', 'conf': "{'spark.executor.memoryOverhead': '5g', 'spark.driver.maxResultSize': '3g', 'spark.driver.memory': '8g', 'spark.driver.extraJavaOptions': '-Xss8m', 'spark.executor.extraJavaOptions': '-Xss8m'}", 'executor_memory': '26g', 'looker_url': 'https://avantstaging.looker.com/dashboards/98', 'campaign_run_name': '202410_001', 'direct_mail_amount_oltp_eligibility_path': 'direct_mail/mailed/eligibility/11107', 'direct_mail_ingestion_file_name': 'direct_mail_ingestion.pq', 'direct_mail_transformation_file_name': 'direct_mail_transformation.pq', 'direct_mail_eligibility_file_name': 'direct_mail_eligibility.pq', 'direct_mail_scoring_file_name': 'direct_mail_scoring.pq', 'direct_mail_approval_file_name': 'direct_mail_approval.pq', 'direct_mail_interim1_path': 'direct_mail/interim1/11107', 'direct_mail_interim2_path': 'direct_mail/interim2/11107', 'direct_mail_first_mail_desirability_path': 'direct_mail/first_mail/desirabiity/11107', 'direct_mail_scoring_full_data_path': 'direct_mail/scoring/11107', 'direct_mail_interim1_final_path': 'direct_mail/interim1_final/11107', 'direct_mail_interim2_final_path': 'direct_mail/interim2_final/11107', 'direct_mail_first_mail_path': 'direct_mail/first_mail/11107', 'direct_mail_first_mail_eligibility_path': 'direct_mail/first_mail/eligibility/11107', 'direct_mail_desirability_file_name': 'direct_mail_desirability.pq', 'direct_mail_treatment_determination_path': 'direct_mail/first_mail/customer_treatment/11107', 'direct_mail_deduped_customer_treatment_path': 'direct_mail/first_mail/deduped_customer_treatment/11107', 'direct_mail_amount_oltp_desirability_path': 'direct_mail/mailed/desirabiity/11107', 'eligibility_source_file_name': 'direct_mail_eligibility_source.pq', 'eligibility_file_name': 'direct_mail_eligibility.pq', 'desirability_path': 'direct_mail/first_mail/desirability/11107', 'eligibility_source_path': 'direct_mail/first_mail/eligibility_source/11107', 'direct_mail_mailed_file_name': 'direct_mail_mailed_ingestion.pq', 'direct_mail_interim_file_name': 'direct_mail_ingestion.pq', 'desirability_file_name': 'direct_mail_desirability.pq', 'transformation_file_name': 'direct_mail_transformation.pq', 'eligibility_path': 'direct_mail/first_mail/eligibility/11107', 'transformation_path': 'direct_mail/first_mail/transformation/11107', 'transformation_dedup_file_name': 'direct_mail_transformation.pq', 'transformation_creative_file_name': 'direct_mail_transformation.pq', 'transformation_creative_path': 'direct_mail/first_mail/creative_code_allocated/11107', 'direct_mail_publish_path': 'preapproved_records/avant/{ENV}', 'transformation_dedup_path': 'direct_mail/first_mail/creative_code_allocated_and_deduped_history/11107', 'scoring_id_columns': 'bpid', 'transformation_final_dedup_path': 'direct_mail/first_mail/creative_code_allocated_and_deduped_final', 'direct_mail_amount_oltp_path': 'direct_mail/mailed/11107', 'direct_mail_publish_file_name': 'direct_mail_publish.csv', 'direct_mail_amount_publish_file_name': 'direct_mail_mailed_11107_{campaign_run_id}_{date}.pq', 'direct_mail_scoring_path': 'direct_mail/first_mail/scoring/11107', 'direct_mail_scoring_history_path': '"avant"."{ENV}"."dw"."dw_orig_history"."direct_mail_model_score"', 'mailed_amount_publish_file_name': 'direct_mail_mailed_11107_{campaign_run_id}_{date}.file_format', 'mailed_publish_file_name': 'direct_mail_mailed_11107_{campaign_run_id}_{date}.csv', 'direct_mail_acxiom_publish_path': 'acxiom/to_acxiom/{ENV}', 'direct_mail_acxiom_publish_file_name': 'AV_MAIL_{batch_key}_SELECTION_{date}.csv', 'selection_dedup_path': 'direct_mail/first_mail/selection_dedup/11107', 'feed_metadata_object': 'common/feed_metadata.cfg', 'uow_dependency_object': 'common/uow_dependency.cfg', 'uow_error_object': 'common/uow_error.cfg', 'config_file_bucket': 'avantcamp-prod-common-bucket', 'config_file_name': 'common/feed_metadata.cfg', 'parameter_config_file': 'common/parameters.cfg', 'acxiom_bucket': 'avant-partner01-landing', 'avant_acxiom_bucket': 'avant-acxiom', 'data_lake_bucket': 'avant-data-lake-ingest', 'publish_bucket': 'avantcamp-partner01-prod-publish-bucket', 'common_bucket': 'avantcamp-partner01-prod-common-bucket', 'exception_bucket': 'avantcamp-partner01-prod-exception-bucket', 'log_bucket': 'avantcamp-partner01-prod-log-bucket', 'selection_bucket': 'avantcamp-partner01-prod-selection-bucket', 'approval_bucket': 'avantcamp-partner01-prod-approval-bucket', 'transient_bucket': 'avantcamp-partner01-prod-transient-bucket', 'ingestion_bucket': 'avantcamp-partner01-prod-ingestion-bucket', 'transform_bucket': 'avantcamp-partner01-prod-transform-bucket', 'scoring_bucket': 'avantcamp-partner01-prod-score-bucket', 'call_logs_amount_publish_path': 'landing/bpo/{ENV}/call_logs', 'bpo_amount_publish_bucket': 'amount-{PARTNER_NAME}-prd-data-lake-ingest', 'call_interval_amount_publish_path': 'landing/bpo/{ENV}/call_interval', 'emails_amount_publish_path': 'landing/bpo/{ENV}/emails', 'qa_amount_publish_path': 'landing/bpo/{ENV}/qa', 'agent_interval_amount_publish_path': 'landing/bpo/{ENV}/agent_interval', 'dremio_ingestion_file_path': '{source_file_type}/{YYYY}/{MM}/{DD}', 'dremio_ingestion_file_name': 'bpo_{source_file_type}_11107_{campaign_run_id}_{date}.{file_format}', 'emails_rename_full_source_path': 'emails/{YYYY}/{MM}/{DD}/bpo_emails_11107_{campaign_run_id}_{date}.{file_format}', 'ivr_call_logs_rename_full_source_path': 'ivr_call_logs/{YYYY}/{MM}/{DD}/bpo_ivr_call_logs_11107_{campaign_run_id}_{date}.{file_format}', 'qa_rename_full_source_path': 'qa/{YYYY}/{MM}/{DD}/bpo_qa_11107_{campaign_run_id}_{date}.{file_format}', 'call_logs_rename_full_source_path': 'call_logs/{YYYY}/{MM}/{DD}/bpo_call_logs_11107_{campaign_run_id}_{date}.{file_format}', 'agent_interval_rename_full_source_path': 'agent_interval/{YYYY}/{MM}/{DD}/bpo_agent_interval_11107_{campaign_run_id}_{date}.{file_format}', 'call_interval_rename_full_source_path': 'call_interval/{YYYY}/{MM}/{DD}/bpo_call_interval_11107_{campaign_run_id}_{date}.{file_format}', 'emails_rename_file_name': 'bpo_emails_11107_{campaign_run_id}_{date}.{file_format}', 'ivr_call_logs_rename_file_name': 'bpo_ivr_call_logs_11107_{campaign_run_id}_{date}.{file_format}', 'qa_rename_file_name': 'bpo_qa_11107_{campaign_run_id}_{date}.{file_format}', 'call_logs_rename_file_name': 'bpo_call_logs_11107_{campaign_run_id}_{date}.{file_format}', 'agent_interval_rename_file_name': 'bpo_agent_interval_11107_{campaign_run_id}_{date}.{file_format}', 'call_interval_rename_file_name': 'bpo_call_interval_11107_{campaign_run_id}_{date}.{file_format}', 'eligibility_selection_history_dedup_path': 'direct_mail/first_mail/eligibility_selection_history_dedup/11107', 'eligibility_selection_latest_dedup_path': 'direct_mail/first_mail/eligibility_selection_dedup/', 'eligibility_dedupe_source_path': 'direct_mail/first_mail/eligibility_dedup_source/', 'direct_mail_amount_latest_send_oltp_path': 'direct_mail/mailed_latest_send', 'direct_mail_scoring_output_path': '{stage_id}/direct_mail/first_mail/11107', 'eligibility_marketing_selection_file_name': 'direct_mail_marketing_eligibility.pq', 'desirability_marketing_selection_file_name': 'direct_mail_marketing_desirability.pq', 'marketing_selection_transformation_creative_file_name': 'direct_mail_marketing_selection_transformation.pq', 'args_dremio_jdbc_user_prod': 'app/dremio/jdbc_prod_sa', 'args_dremio_url_prod': 'https://dremio-prod.services.global.avant.com', 'publish_eloan_bucket': 'avantcamp-partner02-prod-publish-bucket', 'transform_eloan_bucket': 'avantcamp-partner02-prod-transform-bucket', 'ingestion_eloan_bucket': 'avantcamp-partner02-prod-ingestion-bucket', 'args_dremio_vault_path_prod': 'user/devops/dremio/sa_prod_user', 'args_dremio_jdbc_url_prod': 'jdbc:dremio:direct=dremio-int-prod.services.global.avant.com:31010', 'td_dlds_raw_bucket': 'avantdlds-partner03-prod-raw-bucket', 'ingestion_td_bucket': 'avantcamp-partner03-prod-ingestion-bucket', 'td_dlds_transform_bucket': 'avantdlds-partner03-prod-transform-bucket', 'transform_td_bucket': 'avantcamp-partner03-prod-transform-bucket', 'campaign_batch_id': '11107', 'campaign_run_id': '1', 'dremio_data': {4: {'transformation': [{'dremio_config_id': 1, 'campaign_type_config_id': 2, 'stage_id': 4, 'sequence': 1, 'module_config_type': 'transformation', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-transform-bucket.{PARTNER_NAME}.direct_mail.first_mail.creative_code_allocated_and_deduped_final', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_prospect_selected', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}, {'dremio_config_id': 2, 'campaign_type_config_id': 2, 'stage_id': 4, 'sequence': 2, 'module_config_type': 'transformation', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-transform-bucket.{PARTNER_NAME}.direct_mail.first_mail.creative_code_allocated_and_deduped_history', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_prospect_selected_by_send', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}, {'dremio_config_id': 14, 'campaign_type_config_id': 2, 'stage_id': 4, 'sequence': 5, 'module_config_type': 'transformation', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-ingestion-bucket.{PARTNER_NAME}.direct_mail.first_mail.eligibility_source', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_prospect_by_send_campaign', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}, {'dremio_config_id': 15, 'campaign_type_config_id': 2, 'stage_id': 4, 'sequence': 6, 'module_config_type': 'transformation', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-ingestion-bucket.{PARTNER_NAME}.direct_mail.first_mail.eligibility_dedup_source', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_prospect_campaign', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}, {'dremio_config_id': 18, 'campaign_type_config_id': 2, 'stage_id': 4, 'sequence': 9, 'module_config_type': 'transformation', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-selection-bucket.{PARTNER_NAME}.direct_mail.first_mail.eligibility_selection_history_dedup', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_eligible_by_send', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}, {'dremio_config_id': 19, 'campaign_type_config_id': 2, 'stage_id': 4, 'sequence': 10, 'module_config_type': 'transformation', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-selection-bucket.{PARTNER_NAME}.direct_mail.first_mail.eligibility_selection_dedup', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_eligible', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}], 'scoring': [{'dremio_config_id': 5, 'campaign_type_config_id': 2, 'stage_id': 4, 'sequence': 4, 'module_config_type': 'scoring', 're_create': '0.0', 'sql_type': 'custom', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-score-bucket.{PARTNER_NAME}.direct_mail.first_mail.scoring', 'intermediate_vds': '{PARTNER_NAME}.{ENV}.intermediate.dw.dw_orig_history.direct_mail_model_score_by_send', 'intermediate_vds_sql': 'select direct_mail.*Ã¢â‚¬Â¨ from {direct_mail_scoring_history_path} direct_mailÃ¢â‚¬Â¨ INNER JOINÃ¢â‚¬Â¨ (Ã¢â‚¬Â¨select max(etl_ingestion_time) as etl_ingestion_timeÃ¢â‚¬Â¨ from {direct_mail_scoring_history_path}) direct_mail_by_send Ã¢â‚¬Â¨on direct_mail.etl_ingestion_time=direct_mail_by_send.etl_ingestion_time', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_model_score_by_send', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}, {'dremio_config_id': 6, 'campaign_type_config_id': 2, 'stage_id': 4, 'sequence': 3, 'module_config_type': 'scoring', 're_create': '0.0', 'sql_type': 'custom', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-score-bucket.{PARTNER_NAME}.direct_mail.first_mail.scoring', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig_history.direct_mail_model_score', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}]}, 6: {'ingestion': [{'dremio_config_id': 16, 'campaign_type_config_id': 2, 'stage_id': 6, 'sequence': 7, 'module_config_type': 'ingestion', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-ingestion-bucket.{PARTNER_NAME}.direct_mail.mailed', 'intermediate_vds': '{PARTNER_NAME}.{ENV}.intermediate.dw.dw_orig.direct_mail_prospect_mailed_by_send_campaign', 'intermediate_vds_sql': 'select * from avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-ingestion-bucket.{PARTNER_NAME}.direct_mail.mailed', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_prospect_mailed_by_send_campaign', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}, {'dremio_config_id': 17, 'campaign_type_config_id': 2, 'stage_id': 6, 'sequence': 8, 'module_config_type': 'ingestion', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-ingestion-bucket.{PARTNER_NAME}.direct_mail.mailed_latest_send', 'intermediate_vds': '{PARTNER_NAME}.{ENV}.intermediate.dw.dw_orig.direct_mail_prospect_mailed_campaign', 'intermediate_vds_sql': 'select * from avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-ingestion-bucket.{PARTNER_NAME}.direct_mail.mailed_latest_send', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_prospect_mailed_campaign', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-06-04', 'is_active_flag': True}]}, 7: {'transformation': [{'dremio_config_id': 28, 'campaign_type_config_id': 2, 'stage_id': 7, 'sequence': 6, 'module_config_type': 'transformation', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-transform-bucket.{PARTNER_NAME}.direct_mail.first_mail.creative_code_allocated_and_deduped_history', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_prospect_selected_by_send', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-09-16', 'is_active_flag': True}, {'dremio_config_id': 27, 'campaign_type_config_id': 2, 'stage_id': 7, 'sequence': 5, 'module_config_type': 'transformation', 're_create': '0.0', 'sql_type': 'select', 'pds': 'avantcamp-{PARTNER_ID}-{DREMIO_ENV}.avantcamp-{PARTNER_ID}-{ENV}-transform-bucket.{PARTNER_NAME}.direct_mail.first_mail.creative_code_allocated_and_deduped_final', 'intermediate_vds': '', 'intermediate_vds_sql': '', 'final_vds': '{PARTNER_NAME}.{ENV}.dw.dw_orig.direct_mail_prospect_selected', 'final_vds_sql': '', 'created_by': 'avantadmin', 'created_date': '2021-09-16', 'is_active_flag': True}]}}, 'from_date': '2024-08-24', 'to_date': '2024-08-25', 'current_date': '2024-08-25', '4': {'selection': {'eligibility': {'source_path': 's3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/first_mail/eligibility_source/11107/direct_mail_eligibility_source.pq', 'target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/eligibility/11107/direct_mail_eligibility.pq'}, 'desirability': {'source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/eligibility/11107/direct_mail_eligibility.pq', 'target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/desirability/11107/direct_mail_desirability.pq'}, 'crossover': {'source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/desirability/11107/direct_mail_desirability.pq', 'target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/desirability/11107/direct_mail_desirability.pq'}, 'selection_dedup': {'source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/desirability/11107/direct_mail_desirability.pq', 'target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/selection_dedup/11107/direct_mail_desirability.pq', 'eligibility_source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/eligibility/11107/direct_mail_eligibility.pq', 'eligibility_history_target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/eligibility_selection_history_dedup/11107/direct_mail_eligibility.pq', 'eligibility_dedupe_target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/eligibility_selection_dedup//direct_mail_eligibility.pq'}}, 'transformation': {'creative_code_allocation': {'source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/selection_dedup/11107/direct_mail_desirability.pq', 'target_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated/11107/direct_mail_transformation.pq'}, 'dedup_after_creative_code_allocation': {'source_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated/11107/direct_mail_transformation.pq', 'target_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated_and_deduped_history/11107/direct_mail_transformation.pq', 'approval_target_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated_and_deduped_final/direct_mail_transformation.pq'}}}, '3': {'scoring': {'export_path': 'avantcamp-partner01-prod-score-bucket/avant/2/{stage_id}/direct_mail/first_mail/11107', 'table_name_1': 'avantcamp-partner01-prod-transform-bucket', 'table_name_2': 'avant/direct_mail/first_mail/11107/direct_mail_transformation.pq/part', 'id_column': 'bpid'}}, '6': {'publish': [{'source_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/mailed/11107/direct_mail_transformation.pq', 'target_path': 's3://avant-acxiom/preapproved_records/avant/{ENV}'}]}, '7': {'selection': {'eligibility': {'source_path': 's3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/first_mail/eligibility_source/11107/direct_mail_eligibility_source.pq', 'target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/eligibility/11107/direct_mail_marketing_eligibility.pq'}, 'desirability': {'source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/eligibility/11107/direct_mail_marketing_eligibility.pq', 'target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/desirability/11107/direct_mail_marketing_desirability.pq'}, 'crossover': {'source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/desirability/11107/direct_mail_marketing_desirability.pq', 'target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/desirability/11107/direct_mail_marketing_desirability.pq'}, 'selection_dedup': {'source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/desirability/11107/direct_mail_marketing_desirability.pq', 'marketing_selection_source_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated_and_deduped_final/direct_mail_transformation.pq', 'target_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/selection_dedup/11107/direct_mail_marketing_desirability.pq'}}, 'transformation': {'creative_code_allocation': {'source_path': 's3://avantcamp-partner01-prod-selection-bucket/avant/direct_mail/first_mail/selection_dedup/11107/direct_mail_marketing_desirability.pq', 'target_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated/11107/direct_mail_marketing_selection_transformation.pq'}, 'dedup_after_creative_code_allocation': {'source_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated/11107/direct_mail_marketing_selection_transformation.pq', 'target_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated_and_deduped_history/11107/direct_mail_marketing_selection_transformation.pq', 'approval_target_path': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated_and_deduped_final/direct_mail_marketing_selection_transformation.pq'}}}, 'direct_mail_interim1_ingestion': 'avant/direct_mail/interim1/11107/direct_mail_ingestion.pq', 'direct_mail_interim2_ingestion': 'avant/direct_mail/interim2/11107/direct_mail_ingestion.pq', 'mailed_transformation': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/mailed/11107/direct_mail_transformation.pq', 'direct_mail_interim1_parquet_ingestion': 'avant/direct_mail/interim1_final/11107/direct_mail_ingestion.pq', 'direct_mail_interim2_parquet_ingestion': 'avant/direct_mail/interim2_final/11107/direct_mail_ingestion.pq', 'acxiom_joined_with_scoring_file_ingestion': 's3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/first_mail/eligibility_source/11107/direct_mail_eligibility_source.pq', 'acxiom_dedupe_file_ingestion': 's3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/first_mail/eligibility_dedup_source//direct_mail_eligibility_source.pq', 'direct_mail_first_mail_ingestion': 's3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/first_mail/11107/direct_mail_ingestion.pq', 'acxiom_data_transformation': 's3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/first_mail/11107/direct_mail_ingestion.pq', 'direct_mail_first_mail_transformation': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/11107/direct_mail_transformation.pq', 'scoring_file_path': 's3://avantcamp-partner01-prod-score-bucket/avant/direct_mail/first_mail/scoring/11107/direct_mail_scoring.pq', 'scoring_full_data_file': 's3://avantcamp-partner01-prod-score-bucket/avant/direct_mail/scoring/11107/direct_mail_scoring.pq', 'direct_mail_first_mail_deduped_final': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated_and_deduped_final/*', 'direct_mail_first_mail_publish': 's3://avantcamp-partner01-prod-publish-bucket/avant/direct_mail/first_mail/11107/direct_mail_publish.csv', 'direct_mail_amount_oltp_ingestion': 'avant/direct_mail/mailed/11107/direct_mail_mailed_ingestion.pq', 'direct_mail_mailed_latest_send_ingestion': 's3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/mailed_latest_send/direct_mail_mailed_ingestion.pq', 'direct_mail_mailed_ingestion': 's3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/mailed/11107/direct_mail_ingestion.pq', 'direct_mail_mailed_transformation': 's3://avantcamp-partner01-prod-transform-bucket/avant/direct_mail/mailed/11107/direct_mail_transformation.pq', 'direct_mail_remail_publish_file_name': 'acxiom/to_acxiom/{ENV}/AV_MAIL_{batch_key}_SELECTION_{date}.csv', 'file_count': '7000', 'code_name': 'Direct_Mail_2_2_first_mail_ingestion', 'partner_name': 'avant', 'stage_id': '3', 'location': 's3', 'variable_mapping': 'direct_mail', 'model_names': 'avant-production-models/response/en-US/credit_card/1.0', 'shared_vpc_role': 'arn:aws:iam::336913029742:role/AWS-Airflow-Deploy-Role', 'common_service_name': 'Direct_Mail_2_2_first_mail_ingestion', 'scoring_output_files': '10', 'sqs_args': {'sqs_name': '{}_{}_{}_encrypt_decrypt_queue', 'sqs_url': 'https://us-east-2.queue.amazonaws.com/235352882622/DirectMail_2_3_encrypt_decrypt_queue.fifo'}, 'set_processed': 'set_one_file_processed', 'send_number': 189, 'file_list': ['datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_1.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_10.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_11.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_12.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_13.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_14.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_15.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_16.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_17.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_18.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_19.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_2.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_3.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_4.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_5.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_6.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_7.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_8.csv.pgp', 'datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410_9.csv.pgp']}

import sys
import traceback
from campaign_utils.logging_helpers import LoggingHelper
from campaign_utils.vault_client import VaultClient
from campaign_utils.publish import Publish
from campaign_utils.db_helpers import DBHelpers
from campaign_utils.sql_query import SqlQuery
from tempfile import NamedTemporaryFile
import s3fs
from campaign_utils.sqs_helpers import SqsHelpers
from campaign_utils.encrypt_decrypt import EncryptDecrypt
from campaign_utils.s3_helpers import S3Helpers



db_obj = DBHelpers(args)  # noqa: F821
program_path_query = SqlQuery.GET_VAULT_VERSION
program_path_query = program_path_query.format(args['vault_details']['encryption_details'])  # noqa: F821
vault_version = db_obj.execute_query(program_path_query)[0][0]
vault_obj = VaultClient(args)  # noqa: F821
vault_cred = vault_obj.get_credentials(
    secret_path=args['vault_details']['encryption_details'], version=vault_version)  # noqa: F821


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

authenticated
authenticated

In [23]:
encrypt_creds=vault_cred
main_args=args # noqa: F821
_type='decrypt'
file_type='csv'


if _type == 'encrypt' or _type == 'decrypt':
    encrypt_decrypt_obj = EncryptDecrypt(args=encrypt_creds)
else:
    encrypt_decrypt_obj = None
s3_helpers_obj = S3Helpers(args=main_args)
fs = s3fs.S3FileSystem()
sqs_helpers_obj = SqsHelpers(args=main_args)
while True:
    try:
        sqs_message = sqs_helpers_obj.get_sqs_message(_type=file_type)
    except KeyError:
        print("KeyError")
        try:
            sqs_helpers_obj.delete_sqs_queue()
        except sqs_helpers_obj.sqs_resource.meta.client.exceptions.QueueDeletedRecently:
            print("QueueDeletedRecently")
        break
    except sqs_helpers_obj.sqs_resource.meta.client.exceptions.QueueDoesNotExist:
        print("QueueDoesNotExist")
        break

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

using new gpg config
QueueDoesNotExist

In [24]:
if file_type == 'json':
    input_file_data = sqs_message['input_file_data']
    bucket_name = sqs_message['target_bucket_name']
    prefix_path = sqs_message['target_prefix_name']
else:
    sqs_message = sqs_message.split('~')
    input_file_data = sqs_message[0]
    bucket_name = sqs_message[1]
    prefix_path = sqs_message[2]

file_path = input_file_data.split('/')[-1]
file_check = file_path.split('.')[-1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sqs_message' is not defined
Traceback (most recent call last):
NameError: name 'sqs_message' is not defined



In [ ]:
if _type == 'encrypt' or _type == 'decrypt':

    if (
            file_check == 'pq' or
            file_check == 'parquet' or
            file_check == 'csv' or
            file_check == 'pgp' or
            file_check == 'gpg' or
            file_check == 'orc' or
            file_check == 'txt'
    ):
        with fs.open(input_file_data, "rb") as infile:
            with NamedTemporaryFile() as temp_file:
                local_path = temp_file.name
                encrypt_decrypt_obj.encrypt_decrypt_files(
                    infile=infile,
                    local_path=local_path,
                    _type=_type
                )
                s3_helpers_obj.s3_upload_file(
                    local_file=local_path,
                    bucket=bucket_name,
                    object_name=prefix_path.replace('.pgp', '')
                )

In [19]:
# public sftp server
host = 'DOTS.datalabusa.com'
port = 22
username = 'ex_Avant'
password = 'Et#h@ut8Ec2F'

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(hostname=host,port=port,username=username,password=password)
sftp = ssh.open_sftp()

file_list = sftp.listdir('From_Datalab')
campaign_month = '202410' 
campaign_files = [file for file in file_list if campaign_month in file]


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
campaign_files

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['DL_AVT_INTERIMFILE1_202410_1.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_10.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_11.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_12.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_13.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_14.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_15.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_16.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_17.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_18.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_19.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_2.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_3.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_4.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_5.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_6.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_7.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_8.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_9.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_1.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_10.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_11.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_12.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_13.csv.pgp', 'DL_AVT_INTERIMFILE2_2024

In [9]:
list1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['DL_AVT_INTERIMFILE1_202410_1.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_10.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_11.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_12.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_13.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_14.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_15.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_16.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_17.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_18.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_19.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_2.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_3.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_4.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_5.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_6.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_7.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_8.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_9.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_1.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_10.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_11.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_12.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_13.csv.pgp', 'DL_AVT_INTERIMFILE2_2024

In [21]:
file_list

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['archive', 'DL_AVT_INTERIMFILE1_202410_1.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_10.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_11.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_12.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_13.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_14.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_15.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_16.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_17.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_18.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_19.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_2.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_3.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_4.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_5.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_6.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_7.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_8.csv.pgp', 'DL_AVT_INTERIMFILE1_202410_9.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_1.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_10.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_11.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_12.csv.pgp', 'DL_AVT_INTERIMFILE2_202410_13.csv.pgp', 'DL_AVT_INTERI